In [1]:
import pandas as pd
import numpy as np
import pysam

## Section A: Format GENCODE gff3 file

#### 1. Read gff3 file into a pandas dataframe and examine the GENCODE file format

In [2]:
gencode = pd.read_table("/Users/pubudu/Documents/RefData/Gencode/gencode.v19.annotation.gff3", comment="#",
                        sep = "\t", names = ['seqname', 'source', 'feature', 'start' , 'end', 'score', 'strand', 'frame', 'attribute'])
gencode.head()

,seqname,source,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11869,14412,.,+,.,ID=ENSG00000223972.4;gene_id=ENSG00000223972.4...
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ID=ENST00000456328.2;Parent=ENSG00000223972.4;...
2,chr1,HAVANA,exon,11869,12227,.,+,.,ID=exon:ENST00000456328.2:1;Parent=ENST0000045...
3,chr1,HAVANA,exon,12613,12721,.,+,.,ID=exon:ENST00000456328.2:2;Parent=ENST0000045...
4,chr1,HAVANA,exon,13221,14409,.,+,.,ID=exon:ENST00000456328.2:3;Parent=ENST0000045...


In [3]:
gencode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615566 entries, 0 to 2615565
Data columns (total 9 columns):
seqname      object
source       object
feature      object
start        int64
end          int64
score        object
strand       object
frame        object
attribute    object
dtypes: int64(2), object(7)
memory usage: 179.6+ MB


#### 2. Extract Genes in the gff3 file "feature = gene"

In [4]:
gencode_genes = gencode[(gencode.feature == "gene")][['seqname', 'start', 'end', 'attribute']].copy().reset_index().drop('index', axis=1) # Extract genes
gencode_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57820 entries, 0 to 57819
Data columns (total 4 columns):
seqname      57820 non-null object
start        57820 non-null int64
end          57820 non-null int64
attribute    57820 non-null object
dtypes: int64(2), object(2)
memory usage: 1.8+ MB


In [5]:
gencode_genes.head()

,seqname,start,end,attribute
0,chr1,11869,14412,ID=ENSG00000223972.4;gene_id=ENSG00000223972.4...
1,chr1,14363,29806,ID=ENSG00000227232.4;gene_id=ENSG00000227232.4...
2,chr1,29554,31109,ID=ENSG00000243485.2;gene_id=ENSG00000243485.2...
3,chr1,34554,36081,ID=ENSG00000237613.2;gene_id=ENSG00000237613.2...
4,chr1,52473,54936,ID=ENSG00000268020.2;gene_id=ENSG00000268020.2...


#### 3. Extract gene_name, gene_type, gene_status, level of each gene

In [6]:
def gene_info(x):
    # Extract gene names
    g_name = list(filter(lambda x: 'gene_name' in x,  x.split(";")))[0].split("=")[1]
    g_type = list(filter(lambda x: 'gene_type' in x,  x.split(";")))[0].split("=")[1]
    g_status = list(filter(lambda x: 'gene_status' in x,  x.split(";")))[0].split("=")[1]
    g_leve = int(list(filter(lambda x: 'level' in x,  x.split(";")))[0].split("=")[1])
    return (g_name, g_type, g_status, g_leve)

In [7]:
gencode_genes["gene_name"], gencode_genes["gene_type"], gencode_genes["gene_status"], gencode_genes["gene_level"] = zip(*gencode_genes.attribute.apply(lambda x: gene_info(x)))
gencode_genes.head()

,seqname,start,end,attribute,gene_name,gene_type,gene_status,gene_level
0,chr1,11869,14412,ID=ENSG00000223972.4;gene_id=ENSG00000223972.4...,DDX11L1,pseudogene,KNOWN,2
1,chr1,14363,29806,ID=ENSG00000227232.4;gene_id=ENSG00000227232.4...,WASH7P,pseudogene,KNOWN,2
2,chr1,29554,31109,ID=ENSG00000243485.2;gene_id=ENSG00000243485.2...,MIR1302-11,lincRNA,NOVEL,2
3,chr1,34554,36081,ID=ENSG00000237613.2;gene_id=ENSG00000237613.2...,FAM138A,lincRNA,KNOWN,2
4,chr1,52473,54936,ID=ENSG00000268020.2;gene_id=ENSG00000268020.2...,OR4G4P,pseudogene,KNOWN,2


In [8]:
gencode_genes['gene_type'].drop_duplicates()

0                      pseudogene
2                         lincRNA
6                  protein_coding
13                      antisense
14           processed_transcript
15                          snRNA
76                 sense_intronic
82                          miRNA
110                      misc_RNA
254                        snoRNA
315                          rRNA
608      3prime_overlapping_ncrna
614        polymorphic_pseudogene
1274            sense_overlapping
6806                    IG_V_gene
6849                    IG_C_gene
6850                    IG_J_gene
6857              IG_V_pseudogene
21480                   TR_C_gene
21481                   TR_J_gene
21487                   TR_V_gene
21488             TR_V_pseudogene
26189             IG_C_pseudogene
38188                   TR_D_gene
38198             TR_J_pseudogene
39997             IG_J_pseudogene
40004                   IG_D_gene
57783                     Mt_tRNA
57784                     Mt_rRNA
Name: gene_typ

#### 4. Extract all known protein_coding genes

In [9]:
gencode_genes = gencode_genes[gencode_genes['gene_status'] == 'KNOWN'].reset_index().drop('index', axis=1)
gencode_genes = gencode_genes[gencode_genes['gene_type'] == 'protein_coding'].reset_index().drop('index', axis=1)
gencode_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19456 entries, 0 to 19455
Data columns (total 8 columns):
seqname        19456 non-null object
start          19456 non-null int64
end            19456 non-null int64
attribute      19456 non-null object
gene_name      19456 non-null object
gene_type      19456 non-null object
gene_status    19456 non-null object
gene_level     19456 non-null int64
dtypes: int64(3), object(5)
memory usage: 1.2+ MB


#### 5. Remove duplicates - Prioritize verified and manually annotated loci over automatically annotated loci
Gene level column can be used to prioritize genes when removing duplicates 
1. verified loci
2. manually annotated loci
3. automatically annotated loci

In [10]:
## Sort gene_leve in each chromosome (ascending oder) and remove duplicates
gencode_genes = gencode_genes.sort_values(['gene_level', 'seqname'], ascending=True).drop_duplicates('gene_name', keep='first').reset_index().drop('index', axis=1) 
gencode_genes.head()

,seqname,start,end,attribute,gene_name,gene_type,gene_status,gene_level
0,chr1,1246965,1260071,ID=ENSG00000127054.14;gene_id=ENSG00000127054....,CPSF3L,protein_coding,KNOWN,1
1,chr1,1337288,1342693,ID=ENSG00000242485.1;gene_id=ENSG00000242485.1...,MRPL20,protein_coding,KNOWN,1
2,chr1,1353800,1357149,ID=ENSG00000235098.4;gene_id=ENSG00000235098.4...,ANKRD65,protein_coding,KNOWN,1
3,chr1,1550795,1565990,ID=ENSG00000197530.8;gene_id=ENSG00000197530.8...,MIB2,protein_coding,KNOWN,1
4,chr1,1716729,1822495,ID=ENSG00000078369.13;gene_id=ENSG00000078369....,GNB1,protein_coding,KNOWN,1


In [11]:
gencode_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19410 entries, 0 to 19409
Data columns (total 8 columns):
seqname        19410 non-null object
start          19410 non-null int64
end            19410 non-null int64
attribute      19410 non-null object
gene_name      19410 non-null object
gene_type      19410 non-null object
gene_status    19410 non-null object
gene_level     19410 non-null int64
dtypes: int64(3), object(5)
memory usage: 1.2+ MB


* Step 5 removed 46 records (19455-19409)

In [12]:
#### 6. Save the dataframe gencode_genes into a file and index it using Tabix

In [13]:
gencode_genes.to_csv('gencode.v19.annotation.gff3_all_known_genes.txt', index=False, header = False, sep="\t")

In [14]:
%%bash -s gencode.v19.annotation.gff3_all_known_genes.txt
cut -f 1,2,3,5 $1 | sortBed -i > gencode.v19.annotation.gff3_all_known_genes.txt.sorted.formatted.bed
bgzip gencode.v19.annotation.gff3_all_known_genes.txt.sorted.formatted.bed
tabix -p bed gencode.v19.annotation.gff3_all_known_genes.txt.sorted.formatted.bed.gz
ls -l

total 12632
-rw-r--r--  1 pubudu  staff    65214 24 Dec 01:36 01.Format_gencode_gff3.ipynb
-rw-r--r--  1 pubudu  staff       59 20 Dec 20:07 README.md
-rw-r--r--  1 pubudu  staff  6069939 24 Dec 01:36 gencode.v19.annotation.gff3_all_known_genes.txt
-rw-r--r--  1 pubudu  staff   264198 24 Dec 01:36 gencode.v19.annotation.gff3_all_known_genes.txt.sorted.formatted.bed.gz
-rw-r--r--  1 pubudu  staff    61119 24 Dec 01:36 gencode.v19.annotation.gff3_all_known_genes.txt.sorted.formatted.bed.gz.tbi


## Section B: Find genes that overlap with a set of genomic intervals

#### 1. A function to calculate overlapping basepairs

In [15]:
def overlap(q_st, q_end, res_st, res_end):
    o  = min(q_end, res_end)-max(q_st, res_st)
    return o

In [16]:
### Create a random .bed file
!bedtools random -l 100000 -n 50 -g /Users/pubudu/Documents/RefData/hg19/human.hg19.genome | sortBed -i stdin > sample.bed
!head sample.bed

chr1	127569438	127669438	25	100000	-
chr1	194793911	194893911	16	100000	+
chr10	77847895	77947895	44	100000	-
chr10	101647085	101747085	30	100000	+
chr11	50056632	50156632	10	100000	-
chr11	120313124	120413124	37	100000	-
chr12	130593409	130693409	4	100000	-
chr13	15142062	15242062	50	100000	+
chr13	22210089	22310089	14	100000	-
chr13	99558514	99658514	45	100000	-


In [17]:
### Read the sample file in to a pandas dataframe
df = pd.read_table("sample.bed", names=['chr', 'start', 'end', 'name', 'score', 'strand'])
df.head()

,chr,start,end,name,score,strand
0,chr1,127569438,127669438,25,100000,-
1,chr1,194793911,194893911,16,100000,+
2,chr10,77847895,77947895,44,100000,-
3,chr10,101647085,101747085,30,100000,+
4,chr11,50056632,50156632,10,100000,-


#### 2. A function to find overlapping genes in tabix index file

In [18]:
def gencode_all_known_genes(a, tb):
    genes = []

    try:
        for region in tb.fetch(a['chr'], int(a['start']), int(a['end'])):
            if region:
                r = region.split('\t')
                overlap_len = overlap(int(a['start']), int(a['end']), int(r[1]), int(r[2]))
                ret_val = '{}({})'.format(r[3], np.round(overlap_len/float(int(a['end'])-int(a['start']))*100, 2)) ### Percentage of the input interval that overlap with the gene
                genes.append(ret_val) 

        if len(genes)>0:
            return ";".join(genes)
        else:
            return "NA(0)"
    except ValueError:
        return "NA(0)"


In [19]:
import pysam
gencode_v19 = pysam.TabixFile('gencode.v19.annotation.gff3_all_known_genes.txt.sorted.formatted.bed.gz')

df['genes'] = df.apply(lambda x: gencode_all_known_genes(x[['chr', 'start', 'end']], gencode_v19), axis=1)

In [20]:
df.head()

,chr,start,end,name,score,strand,genes
0,chr1,127569438,127669438,25,100000,-,NA(0)
1,chr1,194793911,194893911,16,100000,+,NA(0)
2,chr10,77847895,77947895,44,100000,-,C10orf11(100.0)
3,chr10,101647085,101747085,30,100000,+,DNMBP(100.0)
4,chr11,50056632,50156632,10,100000,-,NA(0)


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
chr       50 non-null object
start     50 non-null int64
end       50 non-null int64
name      50 non-null int64
score     50 non-null int64
strand    50 non-null object
genes     50 non-null object
dtypes: int64(4), object(3)
memory usage: 2.8+ KB


#### When an input interval overlaps with multiple genes, "genes" column will list all those genes (separated by ";")
* If you need to have a single row for each gene, use the following code to transform the dataframe - df

In [22]:
## Remove all the intervals that do not overlap with genes
df = df[df['genes'] != "NA(0)"].reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
chr       30 non-null object
start     30 non-null int64
end       30 non-null int64
name      30 non-null int64
score     30 non-null int64
strand    30 non-null object
genes     30 non-null object
dtypes: int64(4), object(3)
memory usage: 1.7+ KB


In [23]:
new_rows = []
for i,r in df.iterrows():
    g_list = r['genes'].split(";")
    for g in g_list:
        g = g.replace(" ","")
        new_rows.append(np.append(r[['chr', 'start', 'end', 'name', 'score', 'strand', 'genes']].values, g))
        
df_perGene = pd.DataFrame()
df_perGene = df_perGene.append(pd.DataFrame(new_rows, columns=['chr', 'start', 'end', 'name', 'score', 'strand', 'genes', 'gene_ID'])).reset_index().drop('index', axis=1)

df_perGene['gene_name'] = df_perGene['gene_ID'].apply(lambda x: x.split("(")[0])
df_perGene['gene_coverage'] = df_perGene['gene_ID'].apply(lambda x: x.split("(")[1].replace(")", ""))

In [24]:
df_perGene.head()

,chr,start,end,name,score,strand,genes,gene_ID,gene_name,gene_coverage
0,chr10,77847895,77947895,44,100000,-,C10orf11(100.0),C10orf11(100.0),C10orf11,100.0
1,chr10,101647085,101747085,30,100000,+,DNMBP(100.0),DNMBP(100.0),DNMBP,100.0
2,chr11,120313124,120413124,37,100000,-,ARHGEF12(47.52);GRIK4(30.66);AP002348.1(3.22),ARHGEF12(47.52),ARHGEF12,47.52
3,chr11,120313124,120413124,37,100000,-,ARHGEF12(47.52);GRIK4(30.66);AP002348.1(3.22),GRIK4(30.66),GRIK4,30.66
4,chr11,120313124,120413124,37,100000,-,ARHGEF12(47.52);GRIK4(30.66);AP002348.1(3.22),AP002348.1(3.22),AP002348.1,3.22


In [25]:
## drop the genes column
df_perGene = df_perGene.drop(["genes", "gene_ID"], axis=1)
df_perGene.head()

,chr,start,end,name,score,strand,gene_name,gene_coverage
0,chr10,77847895,77947895,44,100000,-,C10orf11,100.0
1,chr10,101647085,101747085,30,100000,+,DNMBP,100.0
2,chr11,120313124,120413124,37,100000,-,ARHGEF12,47.52
3,chr11,120313124,120413124,37,100000,-,GRIK4,30.66
4,chr11,120313124,120413124,37,100000,-,AP002348.1,3.22


In [26]:
df_perGene.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 8 columns):
chr              41 non-null object
start            41 non-null int64
end              41 non-null int64
name             41 non-null int64
score            41 non-null int64
strand           41 non-null object
gene_name        41 non-null object
gene_coverage    41 non-null object
dtypes: int64(4), object(4)
memory usage: 2.6+ KB


## Section C: Find coordinates of a gene list

In [27]:
gencode_genes.head()

,seqname,start,end,attribute,gene_name,gene_type,gene_status,gene_level
0,chr1,1246965,1260071,ID=ENSG00000127054.14;gene_id=ENSG00000127054....,CPSF3L,protein_coding,KNOWN,1
1,chr1,1337288,1342693,ID=ENSG00000242485.1;gene_id=ENSG00000242485.1...,MRPL20,protein_coding,KNOWN,1
2,chr1,1353800,1357149,ID=ENSG00000235098.4;gene_id=ENSG00000235098.4...,ANKRD65,protein_coding,KNOWN,1
3,chr1,1550795,1565990,ID=ENSG00000197530.8;gene_id=ENSG00000197530.8...,MIB2,protein_coding,KNOWN,1
4,chr1,1716729,1822495,ID=ENSG00000078369.13;gene_id=ENSG00000078369....,GNB1,protein_coding,KNOWN,1


In [29]:
gencode_genes = gencode_genes.set_index('gene_name')
gencode_genes.head()

,seqname,start,end,attribute,gene_type,gene_status,gene_level
gene_name,,,,,,,
CPSF3L,chr1,1246965,1260071,ID=ENSG00000127054.14;gene_id=ENSG00000127054....,protein_coding,KNOWN,1
MRPL20,chr1,1337288,1342693,ID=ENSG00000242485.1;gene_id=ENSG00000242485.1...,protein_coding,KNOWN,1
ANKRD65,chr1,1353800,1357149,ID=ENSG00000235098.4;gene_id=ENSG00000235098.4...,protein_coding,KNOWN,1
MIB2,chr1,1550795,1565990,ID=ENSG00000197530.8;gene_id=ENSG00000197530.8...,protein_coding,KNOWN,1
GNB1,chr1,1716729,1822495,ID=ENSG00000078369.13;gene_id=ENSG00000078369....,protein_coding,KNOWN,1


In [30]:
def fetch_gene_coords(g):

    if gencode_genes.index.contains(g): 
        return gencode_genes.loc[g]['seqname'], gencode_genes.loc[g]['start'], gencode_genes.loc[g]['end']  #gencode_genes.loc[g][['seqname', 'start', 'end']]
    else:
        return "NA", "NA", "NA"

In [31]:
df_perGene.head()

,chr,start,end,name,score,strand,gene_name,gene_coverage
0,chr10,77847895,77947895,44,100000,-,C10orf11,100.0
1,chr10,101647085,101747085,30,100000,+,DNMBP,100.0
2,chr11,120313124,120413124,37,100000,-,ARHGEF12,47.52
3,chr11,120313124,120413124,37,100000,-,GRIK4,30.66
4,chr11,120313124,120413124,37,100000,-,AP002348.1,3.22


In [32]:
time df_perGene['g_chr'], df_perGene['g_start'], df_perGene['g_end'] = zip(*df_perGene['gene_name'].apply(lambda x: fetch_gene_coords(x)))

CPU times: user 24.6 ms, sys: 1.98 ms, total: 26.6 ms
Wall time: 25.1 ms


In [33]:
df_perGene.head()

,chr,start,end,name,score,strand,gene_name,gene_coverage,g_chr,g_start,g_end
0,chr10,77847895,77947895,44,100000,-,C10orf11,100.0,chr10,77360998,78319925
1,chr10,101647085,101747085,30,100000,+,DNMBP,100.0,chr10,101635334,101769676
2,chr11,120313124,120413124,37,100000,-,ARHGEF12,47.52,chr11,120207787,120360645
3,chr11,120313124,120413124,37,100000,-,GRIK4,30.66,chr11,120382468,120859613
4,chr11,120313124,120413124,37,100000,-,AP002348.1,3.22,chr11,120382511,120385732
